# Google Analytics - Google Merchandise Store


This is the sample google analytics dataset housed in BigQuery.  It contains Google Analytics 360 data from the Google Merchandise ecommerce Store, which sells Google branded merchandise. 

### Important Information about the dataset

The raw data spans google analytics information from Aug. 2016 to July 2017.  

For the purposes of this project and to save on computer processing power, we will only deal with data from December 1, 2016 to Januar 31, 2017.  

Data in some fields have also been obfuscated or removed for privacy reasons.  More information can be found here: https://support.google.com/analytics/answer/7586738?hl=en#zippy=%2Cin-this-article

So it's important to take results in this data analysis with a grain of salt since we are only seeing data that Google has chosen to make available.  It is not the full story.

## Project Overview and Purpose

This dataset contains 15 columns with several nested arrays. It has a lot of data to explore but at the same time, it is missing a lot of important data on marketing related details such as ads, promotions, campaigns, keyword searches and certain traffic sources due to google's confidentiality concerns and trade secrets.

My main concern is the REVENUE.  Specifically what variables in this dataset have a strong determing factor on the Total Revenue.

So my EDA will be focused on the following areas with several questions to answer on each:

-Revenue & Transactions

-Products

-Traffic Source and Medium (here called 'channelGrouping')

-Visits and Bounces

### Question # 1 - What is the Total Revenue for December 2016 and January 2017?

In [1]:
# importing os library to access Google BigQuery with json key
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/Users/wazabifilms/Documents/Juno Data Analytics Course/CapstoneProjectGA.json'

In [2]:
# accessing specific table in my BigQuery account

import time
from google.cloud import bigquery

query = 'select * from CapstoneProject.TransactionsRevenue'

client = bigquery.Client()
query_job = client.query(query)

time.sleep(2) # wait a bit for the query to finish
result = query_job.result()

### Using SQL to create table in BigQuery

The raw data is too big to import as is in Python so I had to use SQL to query for columns relevant to my questions.  
For the TransactionsRevenue table, the SQL code is as follows:

```sql
SELECT
PARSE_DATE("%Y%m%d", date) AS Date,
 geoNetwork.country as Country,
 totals.transactions as Total_Transactions,
 totals.totalTransactionRevenue/1000000 AS Total_Revenue
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20161201' AND '20170131'
```

In [3]:
# importing pandas library
import pandas as pd

In [4]:
# calling on pandas to set the table as a dataframe
TransactionsRevenue = result.to_dataframe()

In [5]:
# printing first 10 rows to see what the data looks like
TransactionsRevenue.head(10)

,month,continent,transactions,Total_Revenue
0,1,Europe,NaN,NaN
1,1,Europe,NaN,NaN
2,1,Asia,NaN,NaN
3,1,Americas,NaN,NaN
4,1,Americas,NaN,NaN
5,1,Europe,NaN,NaN
6,1,Europe,NaN,NaN
7,1,Asia,NaN,NaN
8,1,Europe,NaN,NaN
9,1,Americas,NaN,NaN


In [6]:
# Trying to see how many rows this has
TransactionsRevenue.shape

(143818, 4)

In [7]:
# replacing the numbers in the 'month' column with the full month year names.
TransactionsRevenue.replace({'month': {1: 'January 2017', 12: 'December 2016'}}, inplace=True)

In [16]:
TransactionsRevenue

,month,continent,transactions,Total_Revenue
0,January 2017,Europe,NaN,NaN
1,January 2017,Europe,NaN,NaN
2,January 2017,Asia,NaN,NaN
3,January 2017,Americas,NaN,NaN
4,January 2017,Americas,NaN,NaN
...,...,...,...,...
143813,December 2016,Americas,NaN,NaN
143814,December 2016,Americas,NaN,NaN
143815,December 2016,Americas,NaN,NaN
143816,December 2016,Americas,NaN,NaN


In [9]:
# creating a new dataframe where I group by month and sum the revenue
Global_Revenue_per_Month = TransactionsRevenue.groupby('month')['Total_Revenue'].sum().reset_index()

### Answer # 1 - Total Revenue Breakdown by Month

In [11]:
Global_Revenue_per_Month

,month,Total_Revenue
0,December 2016,167324.05
1,January 2017,106248.15


Okay, December had 60,000 more in revenue than January.  Makes sense since people usually shop more during Christmas and have less money to spend immediately after New Years.

In [12]:
# creating a new dataframe where I group by continent and sum the revenue
Revenue_per_Continent = TransactionsRevenue.groupby('continent')['Total_Revenue'].sum().reset_index()

### Answer # 1 - Total Revenue Breakdown by Continent for this time period

In [13]:
Revenue_per_Continent

,continent,Total_Revenue
0,(not set),0.00
1,Africa,2007.60
2,Americas,268686.36
3,Asia,2048.94
4,Europe,576.08
5,Oceania,253.22


In [14]:
# creating a pivot table to see the total sum of revenue by month and continent
Revenue_Table = pd.pivot_table(TransactionsRevenue, values='Total_Revenue', index=['month', 'continent'], aggfunc=np.sum)

### Answer # 1 - Total Revenue Breakdown by Month and Continent

In [15]:
Revenue_Table

Total_Revenue
month         continent               
December 2016 (not set)           0.00
              Africa           2007.60
              Americas       163728.80
              Asia             1012.04
              Europe            444.32
              Oceania           131.29
January 2017  (not set)           0.00
              Africa              0.00
              Americas       104957.56
              Asia             1036.90
              Europe            131.76
              Oceania           121.93

Hands down, most of the revenue came from the Americas.  
Either Google is not as popular in the rest of the world OR Google has obfuscated data from the rest of the world in this dataset.  
Or there are many parts of the world that do not use cookies.  
Or goverment regulations in other parts of the world prevent this kind of tracking.
We will just accept that the Americas generated the most revenue in this dataset.

### Question # 2 - What is our Total Transactions per month?  Per Continent?

In [17]:
# creating a new dataframe where I group by month and sum the number of transactions 
Global_Transactions_per_Month = TransactionsRevenue.groupby('month')['transactions'].sum().reset_index()

### Answer # 2 - Total Number of Transactions Breakdown by Month

In [18]:
Global_Transactions_per_Month

,month,transactions
0,December 2016,1450.0
1,January 2017,713.0


This is interesting.  Transactions fell by more than 50% in January from December.  At least that's what it shows in this dataset that may or may not be filtered by Google.

In [20]:
# creating a new dataframe where I group by continent and sum the number of transactions
Transactions_per_Continent = TransactionsRevenue.groupby('continent')['transactions'].sum().reset_index()

### Answer # 2 - Total Number of Transactions Breakdown by Continent for this time period

In [21]:
Transactions_per_Continent

,continent,transactions
0,(not set),0.0
1,Africa,1.0
2,Americas,2125.0
3,Asia,18.0
4,Europe,14.0
5,Oceania,5.0


In [22]:
# creating a pivot table to see the total number of transactions by month and continent
Transactions_Table = pd.pivot_table(TransactionsRevenue, values='transactions', index=['month', 'continent'], aggfunc=np.sum)

### Answer # 2 - Total Number of Transactions by Month and Continent

In [23]:
Transactions_Table 

transactions
month         continent              
December 2016 (not set)           0.0
              Africa              1.0
              Americas         1426.0
              Asia               10.0
              Europe             11.0
              Oceania             2.0
January 2017  (not set)           0.0
              Africa              0.0
              Americas          699.0
              Asia                8.0
              Europe              3.0
              Oceania             3.0

Again, the Americas dominated in the # of transactions.  No surprise based on the fact that this continent generated the most revenue in this time period.

### Question # 3 - What are the Highest Revenue Generating Products for Dec 2016 to Jan 2017?

In [45]:
# accessing specific table in my BigQuery account

from google.cloud import bigquery

prods_query = 'select * from CapstoneProject.ProductsRevised'

client = bigquery.Client()
prods_query_job = client.query(prods_query)

time.sleep(2) # wait a bit for the query to finish
prods_result = prods_query_job.result()

### Using SQL to create 2nd table in BigQuery

For the Products table, the SQL code is as follows:

```sql
SELECT
PARSE_DATE("%Y%m%d", date) AS month,
geoNetwork.continent as continent,
prods.v2ProductName as Product_Name,
prods.productCategory as Product_Category,
prods.productVariant as Product_Variant,
prods.productQuantity as Product_Quantity,
prods.productPrice/1000000 as Product_Price,
prods.productRevenue/1000000 as Product_Revenue
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) as hits,
UNNEST(hits.product) prods
WHERE _TABLE_SUFFIX BETWEEN '20161201' AND '20170131'
```

In [46]:
# calling on pandas to set the table as a dataframe
Products = prods_result.to_dataframe()

In [47]:
# trying to see what the table looks like
Products

,month,continent,Product_Name,Product_Category,Product_Variant,Product_Quantity,Product_Price,Product_Revenue
0,12,Americas,Gift Card - $25.00,Gift Cards,Single Option Only,1,25.00,26.75
1,12,Americas,Gift Card - $25.00,Gift Cards,Single Option Only,1,25.00,26.75
2,12,Americas,Gift Card - $25.00,Gift Cards,Single Option Only,1,25.00,26.75
3,12,Americas,Gift Card - $25.00,Gift Cards,Single Option Only,1,25.00,26.75
4,12,Americas,Google Men's Performance 1/4 Zip Pullover Heat...,Apparel,3XL,1,87.99,91.99
...,...,...,...,...,...,...,...,...
6376,12,Americas,24 oz YouTube Sergeant Stripe Bottle,Drinkware,Single Option Only,1,7.99,9.49
6377,12,Americas,Straw Beach Mat,Lifestyle,Single Option Only,1,7.99,8.24
6378,12,Americas,Red Spiral Google Notebook,Office,Single Option Only,2,7.99,17.18
6379,12,Americas,Red Spiral Google Notebook,Office,Single Option Only,1,7.99,8.24


In [48]:
# trying to find out how many unique products there are
Products['Product_Name'].nunique()

335

In [49]:
# replacing the numbers in the 'month' column with the proper month and year names
Products.replace({'month': {1: 'January 2017', 12: 'December 2016'}}, inplace=True)

### Exploratory Data Analysis on Product Prices, Product Revenue and Quantities of Product Sold in this specific time period

In [50]:
# Generating descriptive statistics on Product price
Products['Product_Price'].describe()

count    6381.000000
mean       18.423659
std        20.660137
min         0.790000
25%         3.990000
50%        13.590000
75%        19.990000
max       250.000000
Name: Product_Price, dtype: float64

In [51]:
Products['Product_Price'].median()

13.59

In [52]:
Products['Product_Price'].mode()

0    13.59
dtype: float64

Okay, so for this period, purchased products cost about 13 to 18 dollars on average.  Pretty affordable.  Now let's see some of the most expensive products that were purchased for this period.

In [80]:
# sorting the dataframe by price in descending order
Products.sort_values(by='Product_Price', ascending=False, inplace=True)

In [82]:
# pulling up the top 5 most expensive products for this time period
Products.head(5)

,month,continent,Product_Name,Product_Category,Product_Variant,Product_Quantity,Product_Price,Product_Revenue
576,December 2016,Americas,Gift Card - $250.00,Gift Cards,Single Option Only,1,250.00,251.500000
85,December 2016,Americas,Google G Noise-reducing Bluetooth Headphones,Electronics,Single Option Only,1,145.99,146.561428
27,January 2017,Americas,Google G Noise-reducing Bluetooth Headphones,Electronics,Single Option Only,1,145.99,146.990000
103,December 2016,Americas,Google Women's Insulated Thermal Vest Navy,Apparel,MD,1,119.99,127.990000
529,December 2016,Americas,Google Men's Performance Full Zip Jacket Black,Apparel,3XL,1,119.99,124.990000


Cool.  The most expensive product that was purchased in this period is a 250 dollar gift card.  

In [53]:
# Generating descriptive statistics on Product revenue
Products['Product_Revenue'].describe()

count    6381.000000
mean       42.922144
std       119.681295
min         1.153636
25%        11.890000
50%        19.990000
75%        45.450000
max      7003.500000
Name: Product_Revenue, dtype: float64

In [54]:
Products['Product_Revenue'].median()

19.99

In [55]:
Products['Product_Revenue'].mode()

0    16.19
dtype: float64

The standard deviation is pretty high, and so is the max.  So this tells me that some products definitely sold way more than others.  Let's see which product generated the most revenue of 7003.5

In [79]:
# Pulling up the row of the max in the Revenue column to see the the product name
Products[Products['Product_Revenue'] == 7003.500000]

,month,continent,Product_Name,Product_Category,Product_Variant,Product_Quantity,Product_Price,Product_Revenue
2131,December 2016,Americas,26 oz Double Wall Insulated Bottle,Drinkware,Single Option Only,350,19.99,7003.5


So the highest revenue generating product for the Dec 2016 to Jan 2017 period is the 26oz Double Wall Insulated Bottle.  Looks like google merch fans are also fans of keeping hydrated.

In [56]:
# Generating descriptive statistics on quantiies sold per product
Products['Product_Quantity'].describe()

count    6381.000000
mean        5.549287
std        19.129536
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max       500.000000
Name: Product_Quantity, dtype: float64

In [57]:
Products['Product_Quantity'].median()

1.0

In [58]:
Products['Product_Quantity'].mode()

0    1
dtype: int64

So most people bought 1 quantity of a product, which makes sense.  Although the average purchase was 5 of a product.  And I'm really wondering what product someone bought 500 of.  Let's take a look at this outlier.

In [76]:
# Pulling up the row with '500' in the column to see the product name
Products[Products['Product_Quantity'] == 500]

,month,continent,Product_Name,Product_Category,Product_Variant,Product_Quantity,Product_Price,Product_Revenue
4651,January 2017,Americas,Maze Pen,Office,Single Option Only,500,0.99,495.75


Okay.  It was 500 pens. Probably for an office before 'work from home' was a thing

In [70]:
# Creating a pivot table for both months 
Products_Table = pd.pivot_table(Products, values='Product_Revenue', index=['continent', 'Product_Name'], aggfunc=np.sum)

In [71]:
# Sorting pivot table by Product Revenue
Products_Table.sort_values(by='Product_Revenue', ascending=False, inplace=True)

### Answer # 3 - These are the top 5 highest revenue generating products for Dec 2016 and Jan 2017 broken down by Continent

In [72]:
# Showing the top 5 products per continent for these 2 months
Products_Table.groupby('continent').head(5)

Product_Revenue
continent Product_Name                                                       
Americas  26 oz Double Wall Insulated Bottle                     11791.387830
          Google Men's  Zip Hoodie                                8037.826611
          Engraved Ceramic Google Mug                             6145.765456
          Waterproof Backpack                                     6023.520547
          Google 22 oz Water Bottle                               4769.153669
Africa    Google Bluetooth Headphones                             2007.600000
Asia      Collapsible Shopping Bag                                 201.250000
          Google Men's Quilted Insulated Vest Battleship ...       181.970000
          Google High Capacity 10,400mAh Charger                   178.636666
          Metal Earbuds with Small Zipper Case                     134.850000
          Google Women's Quilted Insulated Vest Black              121.980000
Europe    Google Men's  Zip Hoodie                                 117.980000
          Google Rucksack                                           71.490000
Oceania   Google Men's Bike Short Sleeve Tee Charcoal               51.970000
Europe    Android Men's  Zip Hoodie                                 50.790000
          Google Tri-blend Hoodie Grey                              36.990000
          Google Men's Long Sleeve Pullover Badge Tee Hea...        33.990000
Oceania   Google Men's Short Sleeve Performance Badge Tee...        23.990000
          Google Men's Skater Tee Grey                              21.990000
          Google Youth Baseball Raglan Heather/Black                20.323333
          Google RFID Journal                                       19.990000

Africa only has 1 product.  Could it be because only 1 purchase or 1 type of product was purchased from there?  

In [83]:
# Trying to see how many purchases were  from Africa
Products[Products['continent'] == 'Africa']

,month,continent,Product_Name,Product_Category,Product_Variant,Product_Quantity,Product_Price,Product_Revenue
55,December 2016,Africa,Google Bluetooth Headphones,Electronics,Single Option Only,40,49.99,2007.6


Yes, only one purchase was made in Africa for this time period.  They bought 40 of them.  I wonder why.  Perhaps to resell?  Or for office equipment perhaps.

### Question # 4 - How many Visitors were there for the Dec. 2016 and Jan 2017 time period?

In [84]:
from google.cloud import bigquery

visits_query = 'select * from CapstoneProject.Visits'

client = bigquery.Client()
visits_query_job = client.query(visits_query)

time.sleep(2) # wait a bit for the query to finish
visits_result = visits_query_job.result()

In [85]:
# calling on pandas to set the table as a dataframe
Visits = visits_result.to_dataframe()

In [86]:
# replacing the numbers in the 'month' column with the proper month and year names
Visits.replace({'month': {1: 'January 2017', 12: 'December 2016'}}, inplace=True)

In [87]:
Visits.head(4)

,month,Visitor_ID,Continent,channelGrouping,visits,bounces
0,January 2017,8579447395603063581,Americas,Organic Search,1,1.0
1,January 2017,5817689849705783870,Asia,Organic Search,1,1.0
2,January 2017,8160804435292640144,Europe,Organic Search,1,1.0
3,January 2017,6209879019453902505,Europe,Organic Search,1,1.0


### Answer # 4 - As per the code and result below, the website had 114,317 unique visitors in this period. 

In [89]:
# Counting the unique number of visitor_Ids
Visits['Visitor_ID'].nunique()

114317

### Question # 5 - What is the breakdown of unique Visitor ID's by month and continent

In [90]:
# Creating a pivot table of COUNT of visitor ids per month and continent
Visitors_by_Month_and_Continent = pd.pivot_table(Visits, values='Visitor_ID', index=['month', 'Continent'], aggfunc=lambda x: len(x.unique()))

### Answer # 5 - See breakdown below for Count of unique Visitors per continent and month

In [91]:
Visitors_by_Month_and_Continent

Visitor_ID
month         Continent            
December 2016 (not set)          73
              Africa           1069
              Americas        32525
              Asia            15143
              Europe          14197
              Oceania           882
January 2017  (not set)         115
              Africa            878
              Americas        26134
              Asia            10341
              Europe          14662
              Oceania           973

### Question # 6 - How many visits were there for this time period?

### Answer # 6 - As per the code and result below, there were 143,818 visits from Dec. 2016 to Jan. 2017.

In [101]:
# summing the visits column
Total_Visits = Visits['visits'].sum()
Total_Visits

143818

## Question # 7 - What is the Breakdown of Global Traffic Source for this time period?

Traffic is the word used to describe visitors/visits coming to a website from a particular virtual origin.  This dataset has already grouped them into 8 categories 

In [97]:
# grouping by channelGrouping and counting unique visitor Id's for each group
Visits.groupby('channelGrouping')['Visitor_ID'].nunique().reset_index()

,channelGrouping,Visitor_ID
0,(Other),4
1,Affiliates,1769
2,Direct,21400
3,Display,1001
4,Organic Search,50707
5,Paid Search,4702
6,Referral,12259
7,Social,25634


### Answer # 7 - As per the above breakdown of global traffic source for this time period...
...most traffic was from organic searches (clicking on a link from a search result), followed by social (traffic coming from social channels like facebook, twitter etc) and then direct (typing the website directly on their browser) 

### Question # 8 - What is the Global Bounce Rate for this time period for each Channel Grouping?

Bounce means when a visitor arrives on the webpage and leaves without going to other pages on the same site.  

Bounce rate is the percentage of visitors who enter the site and then leave rather than continuing to view other pages within the same site

In [98]:
# summing the total bounces
Total_Bounces = Visits['bounces'].sum()     

In [99]:
Total_Bounces

66821.0

In [102]:
# calculating the rate
Bounce_Rate = Total_Bounces/Total_Visits

In [103]:
# rounding to 2 decimal points
Bounce_Rate.round(2)

0.46

The whole website has a 46% bounce rate, which is about average for websites

In [105]:
# creating a table grouping by channelGrouping and a count of their visits
ChannelGroupingsByVisits = Visits.groupby('channelGrouping')['visits'].sum().reset_index()
ChannelGroupingsByVisits

,channelGrouping,visits
0,(Other),6
1,Affiliates,2185
2,Direct,27629
3,Display,1487
4,Organic Search,60777
5,Paid Search,6164
6,Referral,18546
7,Social,27024


In [107]:
# creating a table grouping by channelGrouping and a count of their bounces
ChannelGroupingsByBounces = Visits.groupby('channelGrouping')['bounces'].sum().reset_index()
ChannelGroupingsByBounces

,channelGrouping,bounces
0,(Other),4.0
1,Affiliates,1185.0
2,Direct,13459.0
3,Display,532.0
4,Organic Search,28389.0
5,Paid Search,2519.0
6,Referral,4183.0
7,Social,16550.0


In [108]:
# merging the visit count and bounce count table so we can see both metrics for each channelGrouping
MergedVisitBounce = pd.merge(ChannelGroupingsByVisits, ChannelGroupingsByBounces, left_on = 'channelGrouping', right_on = 'channelGrouping', how = 'left')
MergedVisitBounce

,channelGrouping,visits,bounces
0,(Other),6,4.0
1,Affiliates,2185,1185.0
2,Direct,27629,13459.0
3,Display,1487,532.0
4,Organic Search,60777,28389.0
5,Paid Search,6164,2519.0
6,Referral,18546,4183.0
7,Social,27024,16550.0


In [109]:
# creating another column to the table of the bounce rate per channelGrouping
MergedVisitBounce['Bounce_Rate_Per_Acquisition_Channel'] = (MergedVisitBounce['bounces'] / MergedVisitBounce['visits']) 
MergedVisitBounce

,channelGrouping,visits,bounces,Bounce_Rate_Per_Acquisition_Channel
0,(Other),6,4.0,0.666667
1,Affiliates,2185,1185.0,0.542334
2,Direct,27629,13459.0,0.487133
3,Display,1487,532.0,0.357767
4,Organic Search,60777,28389.0,0.467101
5,Paid Search,6164,2519.0,0.408663
6,Referral,18546,4183.0,0.225547
7,Social,27024,16550.0,0.612419


### Answer # 8 - As per the above breakdown of bounce rate per traffic source...
...the highest bounce rate is for 'other', which according to the metadata, are Partner sites, followed by Social and Affiliates.  The lowest bounce rate is referral, which means another website referred them, so they already had specific reasons for visiting.

## Regression Model to achieve our purpose...

I am interested in seeing if any of the variables studied so far, are good indicators of Total Revenue.  That being said, marketing and advertising are big drivers of revenue.  That data is unfortunately not included in this obfuscated dataset.  Columns relating to ads, marketing campaigns, promotions and keyword search are all null - probably because Google did not want to disclose that information.

So we will work with what we have.  But I don't think we will have a good model.  Let's give it a try nonetheless.

In [110]:
# accessing specific table in my BigQuery account

import time
from google.cloud import bigquery

query = 'select * from CapstoneProject.StatsModelRevised'

client = bigquery.Client()
query_job = client.query(query)

time.sleep(2) # wait a bit for the query to finish
result = query_job.result()

In [111]:
# calling on pandas to set the table as a dataframe
StatsTable = result.to_dataframe()

In [112]:
StatsTable.head(5)

,Date,Continent,visitId,channelGrouping,visits,pageviews,bounces,Total_Transactions,v2ProductCategory,ProductPrice,ProductRevenue,Total_Revenue
0,2016-12-29,Europe,1483004515,Organic Search,1,1.0,1.0,NaN,Home/Shop by Brand/YouTube/,34990000,NaN,NaN
1,2016-12-29,Europe,1483004515,Organic Search,1,1.0,1.0,NaN,Home/Shop by Brand/YouTube/,23990000,NaN,NaN
2,2016-12-29,Europe,1483004515,Organic Search,1,1.0,1.0,NaN,Home/Shop by Brand/YouTube/,24990000,NaN,NaN
3,2016-12-29,Europe,1483004515,Organic Search,1,1.0,1.0,NaN,Home/Shop by Brand/YouTube/,21990000,NaN,NaN
4,2016-12-29,Europe,1483004515,Organic Search,1,1.0,1.0,NaN,Home/Shop by Brand/YouTube/,24990000,NaN,NaN


In [113]:
# Importing the numpy and statsmodels libraries to create a linear regression model
import numpy as np
import statsmodels.api as sm

# Creating a column of 1's in our dataframe to act as a constant.  We will name this column 'intercept'
StatsTable['intercept'] = np.ones((StatsTable.shape[0],))

In [114]:
# Checking to see if the 'intercept' column has been added
StatsTable.head(2)

,Date,Continent,visitId,channelGrouping,visits,pageviews,bounces,Total_Transactions,v2ProductCategory,ProductPrice,ProductRevenue,Total_Revenue,intercept
0,2016-12-29,Europe,1483004515,Organic Search,1,1.0,1.0,NaN,Home/Shop by Brand/YouTube/,34990000,NaN,NaN,1.0
1,2016-12-29,Europe,1483004515,Organic Search,1,1.0,1.0,NaN,Home/Shop by Brand/YouTube/,23990000,NaN,NaN,1.0


In [115]:
# removing unnecessary columns from dataframe;  
# VisitID doesn't make sense to factor in because there will be too many of them and don't provide further insight on individual id's
# The 'visits' column is all 1's since it counts visits per visitorId (according to the metadata)
# The 'bounces' column is also all 1's according to the metadata so it won't make sense in our regression model.
StatsTable.drop(['visitId', 'visits', 'bounces'], axis=1, inplace=True)

In [116]:
# checking to see if columns were indeed dropped
StatsTable.head(2)

,Date,Continent,channelGrouping,pageviews,Total_Transactions,v2ProductCategory,ProductPrice,ProductRevenue,Total_Revenue,intercept
0,2016-12-29,Europe,Organic Search,1.0,NaN,Home/Shop by Brand/YouTube/,34990000,NaN,NaN,1.0
1,2016-12-29,Europe,Organic Search,1.0,NaN,Home/Shop by Brand/YouTube/,23990000,NaN,NaN,1.0


In [117]:
# trying to find out if there are any missing values in the data
StatsTable.isna().any()

Date                  False
Continent             False
channelGrouping       False
pageviews              True
Total_Transactions     True
v2ProductCategory     False
ProductPrice          False
ProductRevenue         True
Total_Revenue          True
intercept             False
dtype: bool

There's a lot of nan values.  I think it makes sense for all the nan values to be replaced with 0s for these columns.  So I will do that AFTER I create my dependent and independent variables tables

In [118]:
# Re-ordering my columns so that my dep vars is index 0.  This way, it will be easier to separate
StatsTable = StatsTable[['Total_Revenue', 'Date', 'Continent', 'channelGrouping', 'pageviews', 'Total_Transactions', 'v2ProductCategory', 'ProductPrice', 'ProductRevenue', 'intercept']]

In [119]:
# checking to see if the re-ordering worked
StatsTable.head(2)

,Total_Revenue,Date,Continent,channelGrouping,pageviews,Total_Transactions,v2ProductCategory,ProductPrice,ProductRevenue,intercept
0,NaN,2016-12-29,Europe,Organic Search,1.0,NaN,Home/Shop by Brand/YouTube/,34990000,NaN,1.0
1,NaN,2016-12-29,Europe,Organic Search,1.0,NaN,Home/Shop by Brand/YouTube/,23990000,NaN,1.0


In [121]:
# Creating my variables by index location of columns
dependent_vars = StatsTable.iloc[:, 0]
independent_vars = StatsTable.iloc[:, 1:]

In [122]:
# checking to see if the table separation worked
dependent_vars

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
2585938   NaN
2585939   NaN
2585940   NaN
2585941   NaN
2585942   NaN
Name: Total_Revenue, Length: 2585943, dtype: float64

In [123]:
# replacing NaN with 0's
dependent_vars = dependent_vars.fillna(0)

In [124]:
# checking to see if Total_Revenue column is not just all 0's
dependent_vars.max()

7003.5

In [125]:
# replacing NaN with 0's
independent_vars = independent_vars.fillna(0)

In [126]:
# checking to see that this independent_vars table is the way I want it:
independent_vars

,Date,Continent,channelGrouping,pageviews,Total_Transactions,v2ProductCategory,ProductPrice,ProductRevenue,intercept
0,2016-12-29,Europe,Organic Search,1.0,0.0,Home/Shop by Brand/YouTube/,34990000,0.0,1.0
1,2016-12-29,Europe,Organic Search,1.0,0.0,Home/Shop by Brand/YouTube/,23990000,0.0,1.0
2,2016-12-29,Europe,Organic Search,1.0,0.0,Home/Shop by Brand/YouTube/,24990000,0.0,1.0
3,2016-12-29,Europe,Organic Search,1.0,0.0,Home/Shop by Brand/YouTube/,21990000,0.0,1.0
4,2016-12-29,Europe,Organic Search,1.0,0.0,Home/Shop by Brand/YouTube/,24990000,0.0,1.0
...,...,...,...,...,...,...,...,...,...
2585938,2016-12-14,Asia,Direct,5.0,0.0,(not set),0,0.0,1.0
2585939,2016-12-14,Asia,Direct,5.0,0.0,(not set),0,0.0,1.0
2585940,2016-12-14,Asia,Direct,5.0,0.0,(not set),0,0.0,1.0
2585941,2016-12-14,Asia,Direct,5.0,0.0,(not set),0,0.0,1.0


In [127]:
# at the moment, the date column is an object so I will change it to an actual date
independent_vars['Date'] = pd.to_datetime(independent_vars['Date'], errors='coerce')

In [129]:
# and changing the date to month, not by day
independent_vars['month']= independent_vars['Date'].dt.month

In [130]:
independent_vars.head(2)

,Date,Continent,channelGrouping,pageviews,Total_Transactions,v2ProductCategory,ProductPrice,ProductRevenue,intercept,month
0,2016-12-29,Europe,Organic Search,1.0,0.0,Home/Shop by Brand/YouTube/,34990000,0.0,1.0,12
1,2016-12-29,Europe,Organic Search,1.0,0.0,Home/Shop by Brand/YouTube/,23990000,0.0,1.0,12


In [131]:
# One hot encoding the 'month' column so that it actually computes it as 2 different months and not that December(12) is better than January(1)
# I am doing this because Regression Modelling only works with numerics
Date_encoded = pd.get_dummies(independent_vars['month'])

In [132]:
#checking that the encoding worked
Date_encoded.head(2)

,1,12
0,0,1
1,0,1


In [133]:
# One hot encoding the 'Continent' column so that it appears as digits instead of categories
Continent_encoded = pd.get_dummies(independent_vars['Continent'])

In [134]:
#checking that the encoding worked
Continent_encoded.head(2)

,(not set),Africa,Americas,Asia,Europe,Oceania
0,0,0,0,0,1,0
1,0,0,0,0,1,0


In [135]:
# One hot encoding the 'channelGrouping' column so that it appears as digits instead of categories
channelGrouping_encoded = pd.get_dummies(independent_vars['channelGrouping'])

In [137]:
#checking that the encoding worked
channelGrouping_encoded.head(2)

,(Other),Affiliates,Direct,Display,Organic Search,Paid Search,Referral,Social
0,0,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0


In [138]:
# One hot encoding the 'v2ProductCategory' column so that it appears as digits instead of categories
Product_encoded = pd.get_dummies(independent_vars['v2ProductCategory'])

In [139]:
#checking that the encoding worked
Product_encoded.head(2)

,${escCatTitle},${productitem.product.origCatName},(not set),Accessories,Android,Apparel,Backpacks,Bags,Bottles,Bottles/,...,Home/Shop by Brand/YouTube/,Housewares,Lifestyle,Lifestyle/,More Bags,Mugs/,Notebooks & Journals,Office,Waze,Wearables/Men's T-Shirts/
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [140]:
# concatenating Date_encoded to independent_vars
independent_vars = pd.concat([independent_vars, Date_encoded], axis=1)

In [141]:
# concatenating Continent_encoded to independent_vars
independent_vars = pd.concat([independent_vars, Continent_encoded], axis=1)

In [142]:
# concatenating channelGrouping_encoded to independent_vars
independent_vars = pd.concat([independent_vars, channelGrouping_encoded], axis=1)

In [143]:
# concatenating Product_encoded to independent_vars
independent_vars = pd.concat([independent_vars, Product_encoded], axis=1)

In [144]:
# checking to see what the concat dataframe looks like now
independent_vars.head(3)

,Date,Continent,channelGrouping,pageviews,Total_Transactions,v2ProductCategory,ProductPrice,ProductRevenue,intercept,month,...,Home/Shop by Brand/YouTube/,Housewares,Lifestyle,Lifestyle/,More Bags,Mugs/,Notebooks & Journals,Office,Waze,Wearables/Men's T-Shirts/
0,2016-12-29,Europe,Organic Search,1.0,0.0,Home/Shop by Brand/YouTube/,34990000,0.0,1.0,12,...,1,0,0,0,0,0,0,0,0,0
1,2016-12-29,Europe,Organic Search,1.0,0.0,Home/Shop by Brand/YouTube/,23990000,0.0,1.0,12,...,1,0,0,0,0,0,0,0,0,0
2,2016-12-29,Europe,Organic Search,1.0,0.0,Home/Shop by Brand/YouTube/,24990000,0.0,1.0,12,...,1,0,0,0,0,0,0,0,0,0


In [145]:
# It doesn't display all the columns so let's pull a list of column names to see that they're all there
independent_vars.columns

Index([                                          'Date',
                                            'Continent',
                                      'channelGrouping',
                                            'pageviews',
                                   'Total_Transactions',
                                    'v2ProductCategory',
                                         'ProductPrice',
                                       'ProductRevenue',
                                            'intercept',
                                                'month',
                                                      1,
                                                     12,
                                            '(not set)',
                                               'Africa',
                                             'Americas',
                                                 'Asia',
                                               'Europe',
                               

In [146]:
# removing columns that we substituted for dummies from dataframe
independent_vars.drop(['Date', 'Continent', 'channelGrouping', 'v2ProductCategory', 'month', '(not set)'], axis=1, inplace=True)

In [147]:
lin_reg = sm.OLS(dependent_vars, independent_vars) #creating the model
reg_results = lin_reg.fit() # running the model
print(reg_results.summary()) # results of the model

                            OLS Regression Results                            
Dep. Variable:          Total_Revenue   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                 1.124e+04
Date:                Thu, 09 Dec 2021   Prob (F-statistic):               0.00
Time:                        01:48:27   Log-Likelihood:            -1.5844e+07
No. Observations:             2585943   AIC:                         3.169e+07
Df Residuals:                 2585860   BIC:                         3.169e+07
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
                                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------

### Okay, the R-squared values are low as expected, but they're not the lowest I've ever seen.  Let's try to remove the continents, since from our early data analysis, most of the revenue was extremely skewed to the Americas

In [148]:
independent_vars2 = independent_vars.drop(columns=['Africa', 'Americas', 'Asia', 'Europe', 'Oceania', '(Other)'])
lin_reg2 = sm.OLS(dependent_vars, independent_vars2)
regression_results2 = lin_reg2.fit()
print(regression_results2.summary())

                            OLS Regression Results                            
Dep. Variable:          Total_Revenue   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                 1.197e+04
Date:                Thu, 09 Dec 2021   Prob (F-statistic):               0.00
Time:                        01:57:08   Log-Likelihood:            -1.5844e+07
No. Observations:             2585943   AIC:                         3.169e+07
Df Residuals:                 2585865   BIC:                         3.169e+07
Df Model:                          77                                         
Covariance Type:            nonrobust                                         
                                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------

### The R-squared hasn't changed.  Let's remove the months and ChannelGrouping to see if that changes anything

In [151]:
independent_vars3 = independent_vars2.drop(columns=[1, 12, 'Affiliates', 'Direct', 'Display', 'Organic Search', 'Paid Search', 'Referral', 'Social'])
lin_reg3 = sm.OLS(dependent_vars, independent_vars3)
regression_results3 = lin_reg3.fit()
print(regression_results3.summary())

                            OLS Regression Results                            
Dep. Variable:          Total_Revenue   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                 1.315e+04
Date:                Thu, 09 Dec 2021   Prob (F-statistic):               0.00
Time:                        02:05:48   Log-Likelihood:            -1.5849e+07
No. Observations:             2585943   AIC:                         3.170e+07
Df Residuals:                 2585873   BIC:                         3.170e+07
Df Model:                          69                                         
Covariance Type:            nonrobust                                         
                                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------

### The r-squared values are a bit smaller.  We're going the wrong way.  Let's remove all the products to see what that will do

In [152]:
independent_vars = StatsTable.iloc[:, 4:]

In [153]:
independent_vars.head(2)

,pageviews,Total_Transactions,v2ProductCategory,ProductPrice,ProductRevenue,intercept
0,1.0,NaN,Home/Shop by Brand/YouTube/,34990000,NaN,1.0
1,1.0,NaN,Home/Shop by Brand/YouTube/,23990000,NaN,1.0


In [154]:
# dropping one column
independent_vars.drop(['v2ProductCategory'], axis=1, inplace=True)

In [155]:
# replacing NaN with 0's
independent_vars = independent_vars.fillna(0)

In [156]:
independent_vars.head(2)

,pageviews,Total_Transactions,ProductPrice,ProductRevenue,intercept
0,1.0,0.0,34990000,0.0,1.0
1,1.0,0.0,23990000,0.0,1.0


In [157]:
lin_reg4 = sm.OLS(dependent_vars, independent_vars)
regression_results4 = lin_reg4.fit()
print(regression_results4.summary())

                            OLS Regression Results                            
Dep. Variable:          Total_Revenue   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.254
Method:                 Least Squares   F-statistic:                 2.206e+05
Date:                Thu, 09 Dec 2021   Prob (F-statistic):               0.00
Time:                        02:16:33   Log-Likelihood:            -1.5859e+07
No. Observations:             2585943   AIC:                         3.172e+07
Df Residuals:                 2585938   BIC:                         3.172e+07
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
pageviews              0.6477      0

# Review of Model Performance

Okay, I'm tapping out.  This made the r-squared valus worse again.  So as expected, these variables alone do not make a good model for predicting the Total Revenue. They probably need to be paired with marketing, ads and promotion data to yield a decent model

# Recommendation based on analysis

Missing marketing, advertisement and promotion data aside, the only meaningful recommendation I can provide has to do with the products since that is the variable with the most dimensions in this given dataset.

I notice that most of the products on offer and most of the products purchased are skewed a bit to males.  Many of the products are either unisex or specifically labeled for men.  Why not try to offer more products marketed towards women - and make them a bit more 'fun' and 'girly,' but still with the air of 'intelligence and sleek simplicity.'  I may be biased to think that women buy more than men.  But it might be worth a try.  If not to also erase the assumption that women are not into 'tech' as much as males.   Perhaps Google, as a leading tech company should make it a duty to attract more women into the industry.  Offering more products that are more attractive to the average female might be a way to do this. 

# Ethical Impact Statement

Ethics on data usually have to do with identifying personal information on a particular individual when that individual has not given permission for their personal details to be made public.  

Most datasets do not contain actual names of people.  Rather, they are given an ID and you can sometimes infer that this ID could be a particular person if other information about them is provided in the same dataset like what city network they are coming from, paired with their sex, birthday, postal code etc. And then you can go down a rabbit hole and perhaps infer their salary down the line.

This google analytics dataset doesn't really pose any ethical concerns because a lot of it is obfuscated.  Sometimes there are values for a visitor ID's geoNetwork city.  But most of the time, there isn't.  And the geoNetwork city is the most granular piece of data on this dataset.  So it's very difficult to pinpoint specific people with this information.